In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/data_trx.csv')

In [3]:
df_dummy = pd.get_dummies(df, prefix='',columns=['subcat_comercial'])

In [4]:
df_dummy = df_dummy.groupby(['fecha', 'venta_id_crp', 'usuario_id_crp']).sum().reset_index()

# Filtrar Compras con un sólo item
df_dummy = df_dummy[df_dummy.iloc[:,3:].sum(axis='columns') > 1]

df_dummy.iloc[:,3:] = df_dummy.iloc[:,3:].apply(lambda x: [1 if y > 1 else y for y in x])

df_dummy.reset_index(drop=True, inplace=True)

In [5]:
df_dummy

,fecha,venta_id_crp,usuario_id_crp,_AGUA,_ALFAJOR,_ALIÑO,_BARQUILLO,_BARRA DE CEREAL,_BARRAS,_BOLLERIA,...,_QUICHE,_REGALO,_SALAME,_SALSA PASTAS,_SANDWICH,_SMOOTHIE,_SOPAS,_VEGETARIANO,_VINO,_YOGHURT
0,2020-01-25,46697,313517,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-25,46698,321398,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-25,46699,213737,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,46703,604765,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-25,46704,266145,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299465,2022-07-09,981793,3335167,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299466,2022-07-09,981796,164135,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299467,2022-07-09,981799,3297053,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
299468,2022-07-09,981800,3335194,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_dummy.to_csv('dummy_apriori.csv', index=False)

In [12]:
frq_items = apriori(df_dummy.iloc[:,3:], min_support = 0.005, use_colnames = True)

/media/ignacio/DS/venvLinux/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [13]:
frq_items

,support,itemsets
0,0.014379,(_AGUA)
1,0.068989,(_ALFAJOR)
2,0.087030,(_BARQUILLO)
3,0.010883,(_BARRA DE CEREAL)
4,0.008983,(_BARRAS)
...,...,...
398,0.005583,"(_QUESO LAMINADO, _PANADERIA, _QUESO)"
399,0.006087,"(_QUESO LAMINADO, _PANADERIA, _JAMON, _BOLLERIA)"
400,0.005984,"(_PANADERIA, _POSTRE INDIVIDUAL, _MAP, _BOLLERIA)"
401,0.005597,"(_QUESO, _CHARCUTERIA, _COCKTAIL CONGELADO, _G..."


In [21]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth

# Building the model
frq_items = apriori(df_dummy.iloc[:,3:], min_support = 0.005, use_colnames = True)
  
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift")#, min_threshold = 1)
#rules = rules.sort_values(['lift'], ascending =[False])
print(rules.head())

/media/ignacio/DS/venvLinux/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


    antecedents   consequents  antecedent support  consequent support  \
0        (_MAP)       (_AGUA)            0.366394            0.014379   
1       (_AGUA)        (_MAP)            0.014379            0.366394   
2  (_BARQUILLO)    (_ALFAJOR)            0.087030            0.068989   
3    (_ALFAJOR)  (_BARQUILLO)            0.068989            0.087030   
4   (_BOLLERIA)    (_ALFAJOR)            0.230824            0.068989   

    support  confidence      lift  leverage  conviction  
0  0.009333    0.025473  1.771575  0.004065    1.011384  
1  0.009333    0.649094  1.771575  0.004065    1.805630  
2  0.017531    0.201435  2.919832  0.011527    1.165856  
3  0.017531    0.254114  2.919832  0.011527    1.224007  
4  0.023321    0.101034  1.464509  0.007397    1.035647  


In [24]:
frq_items.to_csv('frq_items.csv', index=False)

In [25]:
rules.to_csv('association_rules.csv', index=False)

In [8]:
fpgrowth(df_dummy.iloc[100:101,3:], use_colnames=True)

/media/ignacio/DS/venvLinux/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,1.0,(_MAP)
1,1.0,(_JUGO)
2,1.0,(_HELADOS)
3,1.0,(_GASEOSA)
4,1.0,"(_JUGO, _MAP)"
5,1.0,"(_HELADOS, _MAP)"
6,1.0,"(_GASEOSA, _MAP)"
7,1.0,"(_JUGO, _HELADOS)"
8,1.0,"(_JUGO, _GASEOSA)"
9,1.0,"(_HELADOS, _GASEOSA)"


In [133]:
se = set(itemset_test.loc[max(itemset_test.index),'itemsets'])

In [176]:
itemset_test

,support,itemsets,n_items
0,1.0,(MAP),1
1,1.0,(JUGO),1
2,1.0,(HELADOS),1
3,1.0,(GASEOSA),1
4,1.0,"(MAP, JUGO)",2
5,1.0,"(MAP, HELADOS)",2
6,1.0,"(MAP, GASEOSA)",2
7,1.0,"(HELADOS, JUGO)",2
8,1.0,"(GASEOSA, JUGO)",2
9,1.0,"(GASEOSA, HELADOS)",2


In [136]:
se.remove('GASEOSA')

In [137]:
se

{'HELADOS', 'JUGO', 'MAP'}

In [102]:
df_dummy.columns = [x.replace('_', '') for x in df_dummy.columns]

In [109]:
from mlxtend.frequent_patterns import fpgrowth

itemset_test = fpgrowth(df_dummy.iloc[100:101,3:], use_colnames=True)
itemset_test['n_items'] = itemset_test['itemsets'].apply(lambda x: len(x))
de = itemset_test.merge(rules, left_on='itemsets', right_on='antecedents').sort_values(['n_items', 'consequent support'], ascending=[False, False])

/home/ignacio/miniconda3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [110]:
de[['itemsets', 'consequents', 'consequent support', 'support_y', 'confidence', 'lift', 'leverage', 'conviction']]

,itemsets,consequents,consequent support,support_y,confidence,lift,leverage,conviction
100,"(GASEOSA, JUGO)",(MAP),0.366394,0.004855,0.605077,1.651438,0.001915,1.604379
92,"(MAP, JUGO)",(POSTRE INDIVIDUAL),0.204565,0.013741,0.307710,1.504216,0.004606,1.148991
97,"(MAP, GASEOSA)",(POSTRE INDIVIDUAL),0.204565,0.023839,0.524888,2.565877,0.014548,1.674205
95,"(MAP, JUGO)",(PLATO LIVIANO),0.131963,0.007884,0.176550,1.337872,0.001991,1.054146
99,"(MAP, GASEOSA)",(PLATO LIVIANO),0.131963,0.006154,0.135505,1.026838,0.000161,1.004097
...,...,...,...,...,...,...,...,...
3,(MAP),"(GASEOSA, BARQUILLO)",0.007904,0.005620,0.015338,1.940607,0.002724,1.007550
21,(MAP),"(ENSALADA, BOLLERIA)",0.007747,0.004244,0.011584,1.495234,0.001406,1.003882
27,(MAP),"(BARQUILLO, PLATO LIVIANO)",0.007503,0.004007,0.010937,1.457571,0.001258,1.003471
10,(MAP),"(HELADO, SOPAS)",0.007426,0.004508,0.012304,1.656726,0.001787,1.004938


In [123]:
de['is_subset'] = de['consequents'].apply(lambda x: True if x.issubset(se) else False)


In [127]:
de[:3]

,support_x,itemsets,n_items,antecedents,consequents,antecedent support,consequent support,support_y,confidence,lift,leverage,conviction,is_subset
100,1.0,"(GASEOSA, JUGO)",2,"(GASEOSA, JUGO)",(MAP),0.008024,0.366394,0.004855,0.605077,1.651438,0.001915,1.604379,True
92,1.0,"(MAP, JUGO)",2,"(MAP, JUGO)",(POSTRE INDIVIDUAL),0.044656,0.204565,0.013741,0.307710,1.504216,0.004606,1.148991,False
97,1.0,"(MAP, GASEOSA)",2,"(MAP, GASEOSA)",(POSTRE INDIVIDUAL),0.045417,0.204565,0.023839,0.524888,2.565877,0.014548,1.674205,False


In [131]:
[x for x in de['consequents'].unique()[:3]]

[frozenset({'MAP'}),
 frozenset({'POSTRE INDIVIDUAL'}),
 frozenset({'PLATO LIVIANO'})]

In [141]:
len(se)

3

In [174]:
from random import seed
from random import randint
# seed random number generator
# generate some integers
for _ in range(10):
    value = randint(0, len(se)-1)
    print(value)

0
1
0
1
2
1
0
2
2
2


In [139]:
frozenset({'HELADOS'}).issubset(se)

True

In [87]:
[itemset for itemset in de['consequents']]

[frozenset({'_MAP'}),
 frozenset({'_POSTRE INDIVIDUAL'}),
 frozenset({'_POSTRE INDIVIDUAL'}),
 frozenset({'_PLATO LIVIANO'}),
 frozenset({'_PLATO LIVIANO'}),
 frozenset({'_SOPAS'}),
 frozenset({'_HELADO'}),
 frozenset({'_HELADO'}),
 frozenset({'_BARQUILLO'}),
 frozenset({'_GASEOSA'}),
 frozenset({'_ENSALADA'}),
 frozenset({'_MAP'}),
 frozenset({'_MAP'}),
 frozenset({'_BOLLERIA'}),
 frozenset({'_POSTRE INDIVIDUAL'}),
 frozenset({'_POSTRE INDIVIDUAL'}),
 frozenset({'_POSTRE INDIVIDUAL'}),
 frozenset({'_BOLLERIA', '_PANADERIA'}),
 frozenset({'_PLATO LIVIANO'}),
 frozenset({'_PLATO LIVIANO'}),
 frozenset({'_PLATO LIVIANO'}),
 frozenset({'_JUGO'}),
 frozenset({'_JUGO'}),
 frozenset({'_MAP', '_POSTRE INDIVIDUAL'}),
 frozenset({'_MAP', '_POSTRE INDIVIDUAL'}),
 frozenset({'_SOPAS'}),
 frozenset({'_HELADO'}),
 frozenset({'_HELADO'}),
 frozenset({'_HELADO'}),
 frozenset({'_MAP', '_PANADERIA'}),
 frozenset({'_MAP', '_PANADERIA'}),
 frozenset({'_BARQUILLO'}),
 frozenset({'_BARQUILLO'}),
 frozenset

In [85]:
[itemset if itemset not in list(se) for itemset in de['consequents'].unique() else continue]

SyntaxError: invalid syntax (2516669978.py, line 1)

In [66]:
for i in de.iloc[-5,4]:
    print(i in se)

False
True


In [26]:
rules.shape

(282, 9)

In [27]:
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
213,(_GALLETA SALADA),"(_QUESO, _DIP)",0.068221,0.019695,0.011434,0.167597,8.509694,0.010090,1.177681
208,"(_QUESO, _DIP)",(_GALLETA SALADA),0.019695,0.068221,0.011434,0.580536,8.509694,0.010090,2.221356
209,"(_QUESO, _GALLETA SALADA)",(_DIP),0.032621,0.046746,0.011434,0.350496,7.497905,0.009909,1.467666
212,(_DIP),"(_QUESO, _GALLETA SALADA)",0.046746,0.032621,0.011434,0.244589,7.497905,0.009909,1.280599
247,(_QUESO LAMINADO),"(_JAMON, _PANADERIA)",0.052570,0.037366,0.013781,0.262148,7.015687,0.011817,1.304644
242,"(_JAMON, _PANADERIA)",(_QUESO LAMINADO),0.037366,0.052570,0.013781,0.368811,7.015687,0.011817,1.501026
186,"(_CHARCUTERIA, _GALLETA SALADA)",(_QUESO),0.017528,0.104318,0.012535,0.715184,6.855829,0.010707,3.144774
187,(_QUESO),"(_CHARCUTERIA, _GALLETA SALADA)",0.104318,0.017528,0.012535,0.120166,6.855829,0.010707,1.116657
185,"(_QUESO, _GALLETA SALADA)",(_CHARCUTERIA),0.032621,0.056430,0.012535,0.384277,6.809833,0.010695,1.532459
188,(_CHARCUTERIA),"(_QUESO, _GALLETA SALADA)",0.056430,0.032621,0.012535,0.222143,6.809833,0.010695,1.243647


In [16]:
frq_items.sort_values('support', ascending=False)

,support,itemsets
26,0.394694,(_PANADERIA)
24,0.366394,(_MAP)
4,0.230824,(_BOLLERIA)
31,0.204565,(_POSTRE INDIVIDUAL)
62,0.137212,"(_BOLLERIA, _PANADERIA)"
...,...,...
96,0.010195,"(_EMPANADITA, _SOPAS)"
115,0.010115,"(_JAMON, _MAP)"
139,0.010064,"(_MAP FAMILIAR, _POSTRE FAMILIAR)"
162,0.010034,"(_BOLLERIA, _PANADERIA, _HELADO)"
